In [1]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# Read Worksheet 1 into a DataFrame
box_placement = pd.read_excel("/content/Data_Analyst_Intern_Assessment_Test.xlsx", sheet_name="Box Placement")
box_placement.shape

(2356, 81)

In [3]:
# Read worksheet 2 into a DataFrame
wet_harvest = pd.read_excel("/content/Data_Analyst_Intern_Assessment_Test.xlsx", sheet_name="Wet Harvest")
wet_harvest.shape

(2234, 24)

In [4]:
# Read worksheet 3 into DataFrame
dry_harvest = pd.read_excel("/content/Data_Analyst_Intern_Assessment_Test.xlsx", sheet_name="Dry Harvest")
dry_harvest.shape

(2222, 18)

In [5]:
# Merge DataFrames into one DataFrame
df1 = box_placement.merge(wet_harvest, on="@case_id", how="outer", suffixes=("_boxPlace","_wetHarv"))
df1.shape

(2356, 104)

In [6]:
df = df1.merge(dry_harvest, on="@case_id", how="outer", suffixes=("","_dryHarv"))
df.shape

(2356, 121)

##**Checks for enumerators not entering correct data**

**a)** Crazy box dimensions

In [7]:
# Subset all columns about boxes
box_info = df[df.columns[df.columns.str.contains("box")]]

In [8]:
# Boxes with different dimensions
# Mask boxes not 8 by 5 meters
dim_mask = (box_info['box1_length']!=8.0)|(box_info['box2_length']!=8.0)|(box_info['box1_width']!=5.0)|(box_info['box2_width']!=5.0)
var_dim = box_info[dim_mask]

In [9]:
var_dim.shape

(64, 79)

In [10]:
# Not null box dimensions
dim_mask1 = (box_info['box1_length'].notna())|(box_info['box2_length'].notna())|(box_info['box1_width'].notna())|(box_info['box2_width'].notna())
(var_dim[dim_mask1]
[[
'enumerator_comment_boxPlace',
'success_box1',
'success_box2',
'box1_dim_8_by_5',
'box1_length',
'box1_width',
'box1_dry_weight',
'box1_wet_weight'
]]
)

<ipython-input-10-e10090286ac2>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  (var_dim[dim_mask1]


,enumerator_comment_boxPlace,success_box1,success_box2,box1_dim_8_by_5,box1_length,box1_width,box1_dry_weight,box1_wet_weight
608,Birds feed on crops. Therefore no crop survive...,yes,no,yes,8.0,5.0,NaN,NaN
1193,the farmer also experienced same issue as the ...,yes,no,yes,8.0,5.0,NaN,NaN
1194,the wheat all dried up after the farmer plante...,yes,no,yes,8.0,5.0,NaN,NaN
1286,done,yes,no,yes,8.0,5.0,4.330,5.095
1443,done,yes,no,yes,8.0,5.0,4.275,4.275


**No crazy box dimensions were found**

**b)** False zero yield

In [11]:
# Create mask for zero yield
mask_yield = (
(df['box1_wet_weight']<=0.0)|
(df['box1_wet_weight_confirmation']<=0.0)|
(df['box2_wet_weight']<=0.0)|
(df['box2_wet_weight_confirmation']<=0.0)|
(df['box1_dry_weight']<=0.0)|
(df['box1_dry_weight_confirmation']<=0.0)|
(df['box2_dry_weight']<=0.0)|
(df['box2_dry_weight_confirmation']<=0.0)
)

In [12]:
# Info on yield values
df[['box1_wet_weight',
'box1_wet_weight_confirmation',
'box2_wet_weight',
'box2_wet_weight_confirmation',
'box1_dry_weight',
'box1_dry_weight_confirmation',
'box2_dry_weight',
'box2_dry_weight_confirmation'
]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   box1_wet_weight               2131 non-null   float64
 1   box1_wet_weight_confirmation  2131 non-null   float64
 2   box2_wet_weight               2129 non-null   float64
 3   box2_wet_weight_confirmation  2129 non-null   float64
 4   box1_dry_weight               2096 non-null   float64
 5   box1_dry_weight_confirmation  2096 non-null   float64
 6   box2_dry_weight               2093 non-null   float64
 7   box2_dry_weight_confirmation  2093 non-null   float64
dtypes: float64(8)
memory usage: 147.4 KB


In [13]:
# Update mask for missing yield
mask_yield = (
(df['box1_wet_weight'].isna())|
(df['box1_wet_weight_confirmation'].isna())|
(df['box2_wet_weight'].isna())|
(df['box2_wet_weight_confirmation'].isna())|
(df['box1_dry_weight'].isna())|
(df['box1_dry_weight_confirmation'].isna())|
(df['box2_dry_weight'].isna())|
(df['box2_dry_weight_confirmation'].isna())
)

In [14]:
missing_yield = (df[mask_yield][[
'enumerator_comment_boxPlace',
'success_box1',
'success_box2',
'box1_harvest_possible',
'box2_harvest_possible',
'enumerator_comment_wetHarv',
'enumerator_comment',
'box1_wet_weight',
'box1_wet_weight_confirmation',
'box2_wet_weight',
'box2_wet_weight_confirmation',
'box1_dry_weight',
'box1_dry_weight_confirmation',
'box2_dry_weight',
'box2_dry_weight_confirmation',
'why_unable_to_capture_box1_weight',
'username'
]]
)

In [15]:
# filter for box 2 with value and box 1 null
# Wet harvest
(missing_yield[
(missing_yield['box2_wet_weight'].notna())&
(missing_yield['box1_wet_weight'].isna())
]
)

,enumerator_comment_boxPlace,success_box1,success_box2,box1_harvest_possible,box2_harvest_possible,enumerator_comment_wetHarv,enumerator_comment,box1_wet_weight,box1_wet_weight_confirmation,box2_wet_weight,box2_wet_weight_confirmation,box1_dry_weight,box1_dry_weight_confirmation,box2_dry_weight,box2_dry_weight_confirmation,why_unable_to_capture_box1_weight,username
311,Average,yes,yes,NaN,yes,good,good,NaN,NaN,6.260,6.260,NaN,NaN,5.125,5.125,NaN,nig043
1372,poor germination,yes,yes,NaN,yes,This particular farmer his first Box from wet ...,NaN,NaN,NaN,1.575,1.575,NaN,NaN,NaN,NaN,NaN,NaN
1611,poor cultivation,yes,yes,NaN,yes,inexperience of farmer,inexperience of farmer,NaN,NaN,0.955,0.955,NaN,NaN,0.920,0.920,NaN,nig103


In [16]:
# filter for box 2 with value and box 1 null
# Dry harvest
(missing_yield[
(missing_yield['box2_dry_weight'].notna())&
(missing_yield['box1_dry_weight'].isna())
]
)

,enumerator_comment_boxPlace,success_box1,success_box2,box1_harvest_possible,box2_harvest_possible,enumerator_comment_wetHarv,enumerator_comment,box1_wet_weight,box1_wet_weight_confirmation,box2_wet_weight,box2_wet_weight_confirmation,box1_dry_weight,box1_dry_weight_confirmation,box2_dry_weight,box2_dry_weight_confirmation,why_unable_to_capture_box1_weight,username
311,Average,yes,yes,NaN,yes,good,good,NaN,NaN,6.260,6.260,NaN,NaN,5.125,5.125,NaN,nig043
1611,poor cultivation,yes,yes,NaN,yes,inexperience of farmer,inexperience of farmer,NaN,NaN,0.955,0.955,NaN,NaN,0.920,0.920,NaN,nig103


**Enumerators suspected for not entering correct data due to False Zero Yields were those with username nig043 and nig103**

**c)** Dry weight exceeding wet weight

In [17]:
# mask for weight
mask_weight = (
(df['box1_dry_weight']>df['box1_wet_weight'])|
(df['box1_dry_weight_confirmation']>df['box1_wet_weight_confirmation'])|
(df['box2_dry_weight']>df['box2_wet_weight'])|
(df['box2_dry_weight_confirmation']>df['box2_wet_weight_confirmation'])
)

In [18]:
# Mask DataFrame
df_weight = (df[mask_weight][[
'box1_wet_weight',
'box1_wet_weight_confirmation',
'box2_wet_weight',
'box2_wet_weight_confirmation',
'box1_dry_weight',
'box1_dry_weight_confirmation',
'box2_dry_weight',
'box2_dry_weight_confirmation',
'username'
]])

In [19]:
# Weight difference for box 1
df_weight['box1_diff']=(
df_weight['box1_dry_weight_confirmation']-
df_weight['box1_wet_weight_confirmation'])

In [20]:

# Weight difference for box 2
df_weight['box2_diff']=(
df_weight['box2_dry_weight_confirmation']-
df_weight['box2_wet_weight_confirmation'])

In [21]:
# sort values
sorted_weight = df_weight.sort_values(['box1_diff','box2_diff'])

In [22]:
fig = px.bar(sorted_weight['username'].value_counts().sort_values(),
title='Counts of Incorrect Weight Data',
orientation='h')
fig.update_layout(xaxis_title='Count', yaxis_title='Enumerator username')
fig.show()

In [23]:
fig = px.bar(
sorted_weight.groupby('username')[['box1_diff','box2_diff']].agg('mean').sort_values(['box1_diff','box2_diff']).tail(10),
barmode='group',
orientation='h',
title='Mean difference of Dry weight and Wet weight'
)

fig.show()

**nig099 had the highest count of incorrect data by confirmed dry weight exceeding confirmed wet weight. nig030 had the highest mean difference for the considered metric**

##Distribution of Box Placements

**I deem that GIS software would have been better for this part of the analysis. However, due to insufficient infrastructure the work will be carried out here in the notebook**

In [24]:
# Plot for box placement distribution
fig = px.scatter_geo(df,
                     lat='latitude_boxPlace',
                     lon='longitude_boxPlace',
                     hover_name='success_box1',
                     projection='transverse mercator')

# Show the plot
fig.show()

In [25]:
# Plot for wet harvest distribution
fig = px.scatter_geo(df,
                     lat='latitude_wetHarv',
                     lon='longitude_wetHarv',
                     hover_name='box1_wet_weight',
                     projection='transverse mercator')

# Show the plot
fig.show()

##Average Mt/Ha using dry weight

In [26]:
# Computation without filtering outliers
# create df yield
df_yield = (df[['box1_dim_8_by_5','box2_dim_8_by_5',
'box1_dry_weight_confirmation','box2_dry_weight_confirmation']])

In [27]:
# update df yield by filtering out missing yield
df_yield = df_yield[df_yield['box1_dry_weight_confirmation'].notna()]

In [28]:
# Total yield for box 1 in kg
box1 = df_yield['box1_dry_weight_confirmation'].sum()

In [29]:
# Total yield for box 2 in kg
box2 = df_yield['box2_dry_weight_confirmation'].sum()

In [30]:
# Yield in Mt
total_yield=(box2+box1)/1e9

In [31]:
# Area size
totalArea_m2=(len(df_yield)*2)*(8*5)

In [32]:
# Area in Ha
totalArea_Ha = totalArea_m2/1e4

In [33]:
# Average Mt/Ha
print(f'The average mean yield computed using dry weight: \
{total_yield/totalArea_Ha} Mt/Ha')

The average mean yield computed using dry weight: 1.0423408277671757e-06 Mt/Ha


**Computation considering outliers**

In [34]:
# Masks for outliers
# Mask for incorrect data
# Dry weight > Wet weight'

mask_weight = (
(df['box1_dry_weight']<df['box1_wet_weight'])|
(df['box1_dry_weight_confirmation']<df['box1_wet_weight_confirmation'])|
(df['box2_dry_weight']<df['box2_wet_weight'])|
(df['box2_dry_weight_confirmation']<df['box2_wet_weight_confirmation'])
)

# Mask for missing yield
# update df yield by filtering out missing yield
missing=df['box1_dry_weight_confirmation'].notna()

In [35]:
subset1 = df[missing][mask_weight]

<ipython-input-35-d7675ec50308>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [36]:
# Mask for outliers using statistics
# box 1 interquartile range
q3b1 = subset1['box1_dry_weight_confirmation'].quantile(0.75)
q1b1 = np.quantile(subset1['box1_dry_weight_confirmation'],0.25)
iqr1 = q3b1-q1b1

# box 2 interquartile range
q3b2 = np.quantile(subset1['box2_dry_weight_confirmation'],0.75)
q1b2 = np.quantile(subset1['box2_dry_weight_confirmation'],0.25)
iqr2 = q3b2-q1b2

outliers = (
((subset1['box1_dry_weight_confirmation']>(q1b1-1.5*iqr1))&
(subset1['box1_dry_weight_confirmation']<(q3b1+1.5*iqr1)))|
((subset1['box2_dry_weight_confirmation']>(q1b2-1.5*iqr2))&
(subset1['box2_dry_weight_confirmation']<(q3b2+1.5*iqr2)))
)

In [37]:
# subset outliers
subset = subset1[outliers]

In [38]:
# Total yield for box 1 in kg
subset_box1 = subset['box1_dry_weight_confirmation'].sum()

In [39]:

# Total yield for box 2 in kg
subset_box2 = subset['box2_dry_weight_confirmation'].sum()

In [40]:
# Yield in Mt
subset_total_yield=(subset_box2+subset_box1)/1e9

In [41]:
# Area size
sub_totalArea_m2=(len(df_yield)*2)*(8*5)

# Area in Ha
sub_totalArea_Ha = totalArea_m2/1e4

In [42]:
# Average Mt/Ha using subset
print(f'The average mean yield computed using dry weight: \
{subset_total_yield/sub_totalArea_Ha} Mt/Ha')

The average mean yield computed using dry weight: 8.272573354007633e-07 Mt/Ha


##Major problems affecting crops per district

In [43]:
subset.groupby('farmer_state_district')['box1_problem'].value_counts()

farmer_state_district  box1_problem                               
Adamawa                late_planting                                  33
                       poor_germination                               22
                       drought                                        11
                       poor_germination late_planting                  9
                       weeds                                           8
                                                                      ..
Yobe                   weeds late_planting                             1
Zamfara                drought poor_germination locust_infestation    32
                       drought poor_germination                        6
                       drought                                         5
                       poor_germination                                2
Name: count, Length: 118, dtype: int64

**Consistent problems facing crops per district include late planting, poor germination and drought**

##Main Objective

**Yield per hectare per district**

In [44]:
y_perHaperDist=subset.groupby('farmer_state_district')['box1_dry_weight'].agg('sum')/(len(subset)*40)*1e-5

In [45]:
fig = px.bar(
y_perHaperDist.sort_values(),
orientation='h',
title='Mean Yield per Hectare per District(Mt/Ha/District)'
)

fig.show()